In [23]:
import numpy as np
import pandas as pd
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import re
from unidecode import unidecode

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rosiezhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rosiezhao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rosiezhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
file = pd.read_csv('../../Baseresultats - Liste brute.csv')
print("Csv read successfully!")

Csv read successfully!


In [4]:
file.head(10)

,NO,ORG_NAME,TERRITOIRE,SOUS-TERRITOIRE,SECTEUR,NOM_2018,OBJECTIFS_2018,METHODOLOGIE_2018,RESULTATS_2018,Type de résultat,...,CONCLUSION_2012,DIFFUSION_2012,RESUL_ACTION_2012,NOM_2011,OBJECTIF_2011,METHODOLOGIE_2011,RESULTAT_2011,CONCLUSION_2011,DIFFUSION_2011,RESUL_ACTION_2011
0,4215,La Relance-jeunes et familles (RJF) inc.,Centre-est de l'île,Centre-Sud,Famille,VÉOPOCAS - Projet éducatif - se référer au doc...,IDEM,IDEM,IDEM,Autre,...,Les clubs de la Relance à l'école Jean-Batiste...,-Une rencontre bilan est faite auprès de l'équ...,Grilles d'observation avec des indicateurs pré...,Programme d'aide aux devoirs à l'école St-Anselme,La Relance assure la mise en place de programm...,Une formation portant sur les stratégies d'app...,Afin d'expliquer la moyenne et de justifier l'...,"Outre les objectifs définis au départ, l'équip...",Un rapport est distribué auprès de la directio...,Intrervention auprès des enfants les Clubs La ...
1,2591,Projet TRIP,Centre-est de l'île,Centre-Sud,Jeunes,Projet Transitions,Voir document en annexe \r\r\r,Voir document en annexe,Voir documenrt en annexe,Autre,...,Suite des exemples de résultats:\r5) En 2011-2...,Nos stratégies de diffusion des résultats vise...,Nous utilisons une panoplie de moyens pour mie...,Travail de milieu,Voici les principaux résultats attendus par la...,La méthodologie prisée pour évaluer les retomb...,"Voici quelques résultats obtenus en 2009-2010,...","Suite des exemples de résultats:\r5) En 2010, ...",Nos stratégies de diffusion des résultats vise...,Nous utilisons une panoplie de moyens pour mie...
2,187408,"Spectre de rue, programme de travail de milieu",Centre-est de l'île,Centre-Sud,Jeunes,Le programme TAPAJ,Sous l'impulsion de la direction de Spectre de...,"Comme il s’agit d’une recherche participative,...","À ce stade du projet, cet item est sans objet.",Autre,...,Bien que nos résultats dépassent bien nos atte...,Nous diffusons nos résultats sur notre site we...,Pas mal de facon similaire,Travail de milieu visite de l'organisme,Pour mieux sensibliser la population générale ...,des questionnaires d'évaluation sont remis pou...,Plus de 150 personnes différentes en un an se...,Qu'il y a rien de mieux pour sensibiliser et i...,"Sur notre site web, on va surement faire la pr...",Il y a un suivi important de nos statistiques ...
3,49,Les Grands Frères et Grandes Soeurs du Grand M...,Centre-est de l'île,Le Plateau Mont-Royal,Jeunes,Aucun programme n'a fait l'objet d'une évaluat...,Nous ammorcons une évaluation longitudinale de...,NaN,NaN,Autre,...,Le programme connait une croissance depuis son...,Nous diffusons nos résultats auprès de nos par...,"Auprèes des responsables de programmes, du per...",Jeunes en santé,"Les trois objectifs visés sont une vie active,...","Questionnaires, évaluations périodiques et une...",Nous n'avons pas encore reçu les résultats de ...,Le programme connait une croissance depuis son...,Nous diffusons nos résultats auprès de nos par...,"Auprès des responsables de programmes, du pers..."
4,7929,Suicide-Action Montréal inc.,Centre-nord de l'île,Saint-Michel,Écoute et référence,Service de formation,SAM favorise le développement des connaissance...,Faire une appréciation GLOBALe des compétences...,Les participants doivent remplir une appréciat...,Autre,...,Cette démarche réalisée avec plusieurs partena...,L'outil d'estimation de la dangerosité d'un pa...,Nous évaluons les résultats quantitatifs et qu...,"Révision de l'intervention clinique(approche, ...",1- Détecter davantage les clients les plus à r...,1- État de situation et recension des données ...,1- Amélioration des interventions auprès de la...,Cette démarche réalisée avec plusieurs partena...,L'approche et l'intrument d'estimation d'un pa...,Nous évaluons les résultats quantitatifs et qu...
5,7839,Centre communautaire de loisirs de la Côte-des...,Centre-ouest de l'île,Côte-des-Neiges,Vie de quartier,Analyse de la rentabilité des activités offert...,Au moment de sa démarche de planification str

In [66]:
quant_file = file.loc[file['Type de résultat'] == "Effet quantifié"]
#aggregate data from all years
columns = ['RESULTATS_2018', 'RESULTAT_2016', 'RESULTAT_2015', 'RESULTAT_2014','RESULTAT_2013', 'RESULTAT_2012', 'RESULTAT_2011']
data = pd.concat(file[header] for header in columns)
quant_data = [entry for entry in quant_file['RESULTATS_2018']]

#remove all NaN values
quant_data = [entry for entry in quant_data if str(entry) != 'nan']
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('french'))+'\\b'
quant_data = [re.sub(r"{}".format(stop_re), '', entry) for entry in quant_data]
quant_data = [unidecode(entry) for entry in quant_data]

In [67]:
len(quant_data)

76

In [70]:
#preprocessing

#remove characters except letters and numbers
quant_data = [re.sub(r"[^a-zA-Z0-9%\\.]",' ',entry.lower()) for entry in quant_data]


In [73]:
quant_data_tokenized = [entry.split() for entry in quant_data]

In [74]:
quant_data_tokenized

[['la',
  'moyenne',
  'resultats',
  '25',
  'personnes',
  'repondu',
  'questionnaire',
  'deux',
  'fois',
  'repertoriee',
  'ci',
  'dessous',
  'format',
  'suivant',
  'declaration',
  'resultat',
  'moyenne',
  'ecart',
  'entre',
  '1ier',
  '2ieme',
  'score',
  '1',
  'augmentation',
  '0.79',
  'moyenne',
  '2',
  'augmentation',
  '0.64',
  'moyenne',
  '3',
  'augmentation',
  '0.97',
  'moyenne',
  '4',
  'augmentation',
  '1.10',
  'moyenne',
  '5',
  'augmentation',
  '1.06',
  'moyenne',
  '6',
  'augmentation',
  '1.02',
  'moyenne',
  '7',
  'augmentation',
  '0.22',
  'moyenne',
  '8',
  'diminution',
  '1.18',
  'moyenne',
  '9',
  'diminution',
  '1.38',
  'moyenne',
  '10',
  'diminution',
  '1.27',
  'moyenne',
  '11',
  'diminution',
  '0.82',
  'moyenne',
  '12',
  'diminution',
  '1.40',
  'moyenne',
  'note',
  'un',
  'rapport',
  'presentant',
  'plus',
  'clarte',
  'resultats',
  'ainsi',
  'methodologie',
  'employee',
  'peut',
  'etre',
  'fourni',
